# 파이토치 회귀(Vor.5)

In [17]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader # 메모리가 작은 컴퓨터에서 딥러닝을 원샷에 돌리는 것이 불가능함으로 잘라서 넣기 위함임.
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.linear_model import LinearRegression

plt.rcParams["font.family"] = "D2Coding"
plt.rcParams["axes.unicode_minus"] = False

device = "cuda" if torch.cuda.is_available() else "cpu" # 내가 작은 자원이라도 이용해서 사용하기 위한 방법론임

In [18]:
# pyton의 문법은 자바랑 다름
# Dataset -> extends 상속임

# 상속: 재사용성을 높이기 위해 상위 클래스의 기능을 하위 클래스에 가지도록 강제하는 것 / 
#       강한 바인딩이 걸림 / 계층이 가진 기능을 아래 클래스가 가짐 / 만약 하위 클래스가 상위와 상충되면 오버라이딩으로 재정의함
# 인터페이스: 기본적으로 계약 관계 / 반드시 구현하거나 마커를 사용함.

# 파이썬의 상속은 재사용과는 상관 없음.


# CustomDatasetdms Dataset을 상속하고, 해당 클래스에서 필요한 메서드를 덮어 씁니다.

class CustomDataset(Dataset): # 확장 구문 / 그 안의 특정 메서드를 내쪽에서 구현해야 함. / 파이선은 인터페이스 개념이 없음 
    def __init__(slef, x_tensor, y_tensor): # 가장 소중한 것: "생성자" / 모든 클래스는 생성자부터 만들어야 함 / 참고: ( __ : 던던, 매직 메서드)
        slef.x = x_tensor
        slef.y = y_tensor
        
    #Dataset을 상속받기 위해 필요한 2개의 메서드 __getitem__ / __len__
    def __getitem__(self, index):
        return (self.x[index], self.y[index]) # index의 값을 가져올 때 사용!!
    def __len__(self):
        return len(self.x)
    
    
 #이것만 하면 데이터 set을 만들 수 있음
 # 애를 왜써야 할까?

In [19]:
true_w = 2
true_b = 1
N = 100
np.random.seed(42)
x = np.random.rand(N, 1)
epsilon = 0.1 * np.random.randn(N, 1) # 적당히 작은 노이즈 추가
y = true_b + (true_w * x) + epsilon # 우리가 만든 정답지


idx = np.arange(N)
np.random.shuffle(idx)
train_idx = idx[:int(N *0.8)]
val_idx = idx[int(N*0.8) :]
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

dataset = CustomDataset(x_train_tensor, y_train_tensor)

ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train

train_data, val_data = random_split(dataset, [n_train, n_val])

train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True) # 로드 데이터를 불러 옴.
val_loader = DataLoader(dataset=train_data, batch_size=16) # 로드 데이터를 불러 옴.

# train_data = TensorDataset(x_train_tensor, y_train_tensor) 이거 쓰면 iris, 타이타닉 등 사용 안됨


In [20]:
lr = 0.1
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1, 1)).to(device) # 레이어가 128개 늘어날 텐데 이것도 클래스 같은 것을 써서 관리 하겠구나.. 생각해야 함. 
optimizer = optim.SGD(model.parameters(), lr = lr)
loss_fn = nn.MSELoss(reduction="mean")

# for epoch in range(n_epochs):
#     model.train()
#     yhat = model(x_train_tensor)
#     loss = loss_fn(yhat, y_train_tensor)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()
# print(model.state_dict())


- 학습용
     - 모델은 계속해서 업데이트 하고, 검증은 동일해야 효율이 보임

In [21]:
def make_train_step_fn(model, loss_fn, optimizer): # 안쪽의 함수를 내 뱉으니까 
    def perform_train_step_fn(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return perform_train_step_fn


- 검증용

In [22]:
def make_val_step_fn(model, loss_fn):
    def perform_val_step_fn(x, y):
        model.eval()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        return loss.item()
    return perform_val_step_fn

In [23]:
def mini_batch(device, data_loader, step_fn):
    mini_betch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_betch_losses.append(mini_batch_loss)
    loss = np.mean(mini_betch_losses)
    return loss

In [24]:
n_epochs = 1000
losses = []
train_setp_fn = make_train_step_fn(model, loss_fn, optimizer)
for epoch in range(n_epochs):
    loss = train_setp_fn(x_train_tensor, y_train_tensor)
    losses.append(loss)
    
print(model.state_dict)

<bound method Module.state_dict of Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)>


In [25]:
n_epochs = 200
losses = []
val_losses = []

train_setp_fn = make_train_step_fn(model, loss_fn, optimizer)
val_setp_fn = make_val_step_fn(model, loss_fn)

write = SummaryWriter("run/simple_linear_regression")
x_sample, y_sample = next(iter(train_loader))
write.add_graph(model, x_sample.to(device))

for epoch in range(n_epochs):
    # loss = train_setp_fn(x_train_tensor, y_train_tensor)
    loss = mini_batch(device, train_loader, train_setp_fn)
    losses.append(loss)
    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_setp_fn)
        val_losses.append(val_loss)
    write.add_scalars(
        main_tag="loss",
        tag_scalar_dict={"training":loss, "val_loss": val_loss},
        global_step=epoch
    )
write.close()
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9582]])), ('0.bias', tensor([1.0274]))])


In [30]:
# TensorBoard 확장 로드
%load_ext tensorboard

# 로그 디렉터리 지정 후 실행
%tensorboard --logdir run


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 7312), started 0:02:39 ago. (Use '!kill 7312' to kill it.)

In [27]:
next(iter(train_loader))

[tensor([[0.7296],
         [0.4938],
         [0.5979],
         [0.5427],
         [0.0344],
         [0.6075],
         [0.3309],
         [0.3745],
         [0.1960],
         [0.9696],
         [0.6233],
         [0.7722],
         [0.1159],
         [0.0885],
         [0.8872],
         [0.1196]]),
 tensor([[2.5751],
         [1.9060],
         [2.0407],
         [2.2161],
         [1.1831],
         [2.4037],
         [1.5427],
         [1.7578],
         [1.4393],
         [2.8401],
         [2.2940],
         [2.4208],
         [1.1603],
         [1.0708],
         [2.8708],
         [1.3214]])]

# 고차원 함수(HoF) 연습

In [28]:


# def square(x):
#     return x **2

# def cube(x):
#     return x **3

# def forth_power(x):
#     return x**4

# def generic_expon(x, expon):
#     return x**expon

# 위 처럼 만들지 않는다. 


# 캡슐링
def exponentiation_builder(exponnent):
    def skeleton_exponentiation(x):
        return x ** exponnent
    return skeleton_exponentiation

In [29]:
returned_fn = exponentiation_builder(10)
returned_fn(5)

9765625